# Installation

## (OPTIONAL) Check GPU Availability

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
!nvidia-smi

## Set paths

In [ ]:
import os

In [ ]:
CUSTOM_MODEL_NAME = "ssd_mobilenet"
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        !mkdir -p {path}

## Clone Tensorflow Model Garden and install dependencies for object detection

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd Tensorflow/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

## Test installation

In [ ]:
%%bash
cd Tensorflow/models/research/
python3 object_detection/builders/model_builder_tf2_test.py

# Preparation

## Download dataset

This notebook expects a single archive with the following structure:
```
archive.tar.gz/
  train/
    picture1.png
    picture1.xml
    ...
  test/
    picture1.png
    picture1.xml
    ...
```
If your dataset doesn't match this structure you'll have to manually convert it accordingly.


In [ ]:
# Only needed when importing from Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

## Download pretrained model

In [ ]:
import object_detection

In [ ]:
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

## Create Label Map

In [ ]:
labels = [{'name':'plate', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Create TF records

In [ ]:
%%bash
cd Tensorflow/scripts
wget https://raw.githubusercontent.com/nicknochnack/GenerateTFRecord/main/generate_tfrecord.py

Make sure that the following step correctly fills the .record files. If not you might have to run it twice.

In [ ]:
!python3 {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} -c {os.path.join(paths['ANNOTATION_PATH'], 'train.csv')}
!python3 {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} -c {os.path.join(paths['ANNOTATION_PATH'], 'test.csv')}

In [ ]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

## Update config

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

# Train the model

In [ ]:
cmd = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=500".format(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py'), paths['CHECKPOINT_PATH'], files['PIPELINE_CONFIG'])
print(cmd)

In [ ]:
!{cmd}

# Evaluate the model

In [ ]:
cmd = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py'), paths['CHECKPOINT_PATH'], files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(cmd)

In [ ]:
!{cmd}

## (OPTIONAL) Visualize using Tensorboard

In [ ]:
tb_path = os.path.join(paths['CHECKPOINT_PATH'], 'eval') # Change to 'train' if needed
!cd {tb_path}
%load_ext tensorboard
%tensorboard --logdir .

# Install dependencies for OCR

In [ ]:
!pip3 install "paddleocr>=2.0.1" paddlepaddle

# Run ANPR

In [ ]:
img = '"/content/Tensorflow/workspace/images/test/images (13).jpg"' # Path to image that you want to process

## Run using standard Python script

In [ ]:
!python anpr_detector.py -i {img} -c {CUSTOM_MODEL_NAME} -cp 0

## Run using IPython

Use this if you want to visualize the results.

In [ ]:
%run -i anpr_detector.py -i {img} -c {CUSTOM_MODEL_NAME} -cp 0